In [1]:
import csv
import re

def generate_insert_statements(
    csv_file, 
    create_stream_statement, 
    output_file, 
    table_name,
    encoding='utf-8'
):
    # Extract content within parentheses
    stream_content = re.search(r'\((.*?)\)', create_stream_statement, re.DOTALL).group(1)
    
    # Split content into individual column definitions
    columns = re.findall(r'([a-zA-Z_]+)\s+([a-zA-Z_]+)', stream_content)

    # Extract column names and data types, stripping BOM if present
    column_names_create_stream = [col[0].strip() for col in columns]
    data_types_create_stream = {col[0].strip(): col[1] for col in columns}

    # Open the CSV file
    with open(csv_file, newline='', encoding=encoding, errors='ignore') as csvfile:
        reader = csv.DictReader(csvfile)
        insert_statements = []

        # Iterate over each row in the CSV
        for row in reader:
            columns = []
            values = []

            # Iterate over each column in the row
            for field, value in row.items():
                if field in column_names_create_stream:
                    # Column name exists in create_stream_statement
                    data_type = data_types_create_stream[field]
                    if value is None:
                        columns.append(field)
                        values.append("")
                    else:
                        if data_type == "DOUBLE" or data_type == "FLOAT" or data_type == "INTEGER":
                            value_str = str(value)
                            if value_str.strip() == "":
                                columns.append(field)
                                values.append("")
                            else:
                                columns.append(field)
                                values.append(value_str)
                        elif data_type == "TIMESTAMP" or data_type == "DATE":
                            columns.append(field)
                            values.append("'" + str(value) + "'")
                        elif data_type == "VARCHAR":
                            columns.append(field)
                            values.append("'" + value.replace("'", "''") + "'")
                        # Handle other data types as needed

            # Construct the INSERT INTO statement with the specified table name
            insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"
            insert_statements.append(insert_statement)

    # Write insert statements to output file
    with open(output_file, 'w', encoding=encoding, errors='ignore') as f:
        for statement in insert_statements:
            f.write(statement + '\n')

In [2]:
create_stream_statement = """
CREATE TABLE IF NOT EXISTS dataco_scd_table (
    transaction_type VARCHAR,
    actual_days_to_ship INTEGER,
    scheduled_days_to_ship INTEGER,
    benefit_per_order FLOAT,
    sales_per_customer FLOAT,
    delivery_status VARCHAR,
    late_delivery_risk INTEGER,
    category_id INTEGER,
    category_name VARCHAR,
    customer_city VARCHAR,
    customer_country VARCHAR,
    customer_email VARCHAR,
    customer_fname VARCHAR,
    customer_id INTEGER,
    customer_lname VARCHAR,
    customer_password VARCHAR,
    customer_segment VARCHAR,
    customer_state VARCHAR,
    customer_street VARCHAR,
    customer_zipcode INTEGER,
    department_id INTEGER,
    department_name VARCHAR,
    latitude FLOAT,
    longitude FLOAT,
    market VARCHAR,
    order_city VARCHAR,
    order_country VARCHAR,
    order_customer_id INTEGER,
    order_date VARCHAR,
    order_id INTEGER,
    order_item_cardprod_id INTEGER,
    order_item_discount FLOAT,
    order_item_discount_rate FLOAT,
    order_item_id INTEGER,
    order_item_product_price FLOAT,
    order_item_profit_ratio FLOAT,
    order_item_quantity INTEGER,
    sales FLOAT,
    order_item_total FLOAT,
    order_profit_per_order FLOAT,
    order_region VARCHAR,
    order_state VARCHAR,
    order_status VARCHAR,
    order_zipcode INTEGER,
    product_card_id INTEGER,
    product_category_id INTEGER,
    product_desc VARCHAR,
    product_image VARCHAR,
    product_name VARCHAR,
    product_price FLOAT,
    product_status INTEGER,
    shipping_date VARCHAR,
    shipping_mode VARCHAR
)
"""

input_data_file = '/Users/briandunn/Desktop/Apache_Kafka-Kafka_Connect_non_postgres/DataCO Supply Chain - Postgres to MongoDB/orig-data/DataCoSupplyChainDataset.csv'

output_file_location = '/Users/briandunn/Desktop/Apache_Kafka-Kafka_Connect_non_postgres/DataCO Supply Chain - Postgres to MongoDB/source-scripts/3_inserts.sql'

generate_insert_statements(
    input_data_file, 
    create_stream_statement, 
    output_file_location,
    "dataco_scd_table"
    )

In [3]:
# Replace ", ''," and ", ," with ", NULL,"

def replace_empty_with_null(input_file, output_file):
    # Open the input file in read mode and the output file in write mode
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Iterate over each line in the input file
        for line in infile:
            # Replace occurrences of ", ''," and ", ," with ", NULL,"
            updated_line = line.replace(", '',", ", NULL,").replace(", ,", ", NULL,")
            # Write the updated line to the output file
            outfile.write(updated_line)

In [4]:
# Example usage:
# Replace 'input.csv' with your input file name and 'output.csv' with your desired output file name
input_file = '/Users/briandunn/Desktop/Apache_Kafka-Kafka_Connect_non_postgres/DataCO Supply Chain - Postgres to MongoDB/source-scripts/3_inserts.sql'
output_file = '/Users/briandunn/Desktop/Apache_Kafka-Kafka_Connect_non_postgres/DataCO Supply Chain - Postgres to MongoDB/source-scripts/4_inserts.sql'
replace_empty_with_null(input_file, output_file)

Now, delete the 3_inserts.sql file and then rename the 4_inserts.sql file to 3_inserts.sql